In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
training_path = F"/content/gdrive/My Drive/New Colab Notebooks/task3_training.tsv"
validation_path = F"/content/gdrive/My Drive/New Colab Notebooks/task3_validation.tsv"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression

In [0]:
train_data = pd.read_csv(training_path,sep='\t', encoding="ISO-8859-1")
train_data = train_data.loc[:, ~train_data.columns.str.contains('^Unnamed')]

In [0]:
test_data = pd.read_csv(validation_path,sep='\t', encoding="ISO-8859-1")
test_data = test_data.loc[:, ~test_data.columns.str.contains('^Unnamed')]

In [0]:
train_data = train_data[train_data.type == "ADR"]
test_data = test_data[test_data.type == "ADR"]

In [0]:
X_train = train_data.drop(labels=['meddra_term', 'meddra_code'], axis=1)
Y_train = pd.DataFrame(train_data['meddra_term'])
X_test = test_data.drop(labels=['meddra_term', 'meddra_code'], axis=1)
Y_test = pd.DataFrame(test_data['meddra_term'])

In [0]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
Y_train['meddra_term_category'] = labelencoder.fit_transform(Y_train['meddra_term'])
Y_test['meddra_term_category'] = labelencoder.fit_transform(Y_test['meddra_term'])

In [8]:
X_train.head()

,tweet_id,begin,end,type,extraction,drug,tweet
27,342314998904786945,42.0,53.0,ADR,lost vision,cymbalta,"#cymbalta withdrawal has reached a peak, lost ..."
28,342314998904786945,11.0,21.0,ADR,withdrawal,cymbalta,"#cymbalta withdrawal has reached a peak, lost ..."
29,342322703556038657,27.0,35.0,ADR,nauseous,cipro,i hate cipro! #antibiotic #nauseous #cf #hospi...
30,342349802601844737,109.0,118.0,ADR,can't cum,seroquel,@luckystubbs reppin zoloft&amp;seroquel since ...
31,342349802601844737,101.0,104.0,ADR,fat,seroquel,@luckystubbs reppin zoloft&amp;seroquel since ...


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

logreg = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression(n_jobs=1, C=1e5, max_iter=350)),
              ])
logreg.fit(X_train["extraction"], Y_train["meddra_term"])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=100000.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=

In [10]:
print(Y_test.meddra_term.unique)

<bound method Series.unique of 0                    drug allergy
1                    liver damage
2      attention deficit disorder
3              attention impaired
4                           death
                  ...            
390               hypochondriasis
391           withdrawal syndrome
392                      low mood
393                   weight gain
394      electric shock sensation
Name: meddra_term, Length: 365, dtype: object>


In [11]:
# logmodel = LogisticRegression()
# logmodel.fit(X_train["extraction"], Y_train['meddra_term_category'])
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
y_pred = logreg.predict(X_test["extraction"])
print(classification_report(Y_test["meddra_term"], y_pred))
print (accuracy_score(Y_test['meddra_term'], y_pred))

                                  precision    recall  f1-score   support

                MS-like syndrome       0.00      0.00      0.00         1
                   Schizophrenia       0.00      0.00      0.00         1
                 abnormal dreams       0.00      0.00      0.00         0
                            ache       0.00      0.00      0.00         2
      achilles tendon discomfort       0.00      0.00      0.00         1
                 aching in knees       0.00      0.00      0.00         0
                       addiction       0.00      0.00      0.00         2
             aggressive behavior       0.00      0.00      0.00         1
               allergic reaction       0.50      1.00      0.67         3
                         anxiety       1.00      1.00      1.00         1
                 appetite absent       0.00      0.00      0.00         0
                   appetite lost       0.00      0.00      0.00         0
             appetite suppressed     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
import gensim.downloader as api
model = api.load("glove-twitter-25")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [13]:
model.similarity('tummy', 'stomach')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.91068697

In [14]:
pip uninstall smart_open

Uninstalling smart-open-2.0.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/smart_open-2.0.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/smart_open/*
Proceed (y/n)? y
  Successfully uninstalled smart-open-2.0.0


In [15]:
pip install smart_open

Processing /root/.cache/pip/wheels/27/65/38/8d7f5fe8d7afb4e4566587b2d1933cec185fba19257836c943/smart_open-2.0.0-cp36-none-any.whl


In [16]:
import gensim
from gensim.models import KeyedVectors
w2v = F"/content/gdrive/My Drive/New Colab Notebooks/word2vec_twitter_tokens.bin"
wv = KeyedVectors.load_word2vec_format(w2v, binary=True, unicode_errors='ignore')
wv.similarity('preggers','pregnant')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.75958073

In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
def word_averaging(wv, words):
    all_words, mean = set(), []

    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [0]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [20]:
test_tokenized = test_data.apply(lambda r: w2v_tokenize_text(r['extraction']), axis=1).values
train_tokenized = train_data.apply(lambda r: w2v_tokenize_text(r['extraction']), axis=1).values

X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [21]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5,max_iter=300)
logreg = logreg.fit(X_train_word_average, Y_train["meddra_term"])
y_pred = logreg.predict(X_test_word_average)
print(classification_report(Y_test["meddra_term"], y_pred))
print('accuracy %s' % accuracy_score(y_pred, Y_test["meddra_term"]))

                                 precision    recall  f1-score   support

               MS-like syndrome       0.00      0.00      0.00         1
                  Schizophrenia       0.00      0.00      0.00         1
                abnormal dreams       0.00      0.00      0.00         0
                           ache       0.00      0.00      0.00         2
     achilles tendon discomfort       0.00      0.00      0.00         1
                aching in knees       0.00      0.00      0.00         0
                      addiction       0.00      0.00      0.00         2
            aggressive behavior       0.00      0.00      0.00         1
                       agitated       0.00      0.00      0.00         0
              allergic reaction       0.50      1.00      0.67         3
                        anxiety       1.00      1.00      1.00         1
                appetite absent       0.00      0.00      0.00         0
                  appetite lost       0.00      0.

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in

In [25]:
# SVM
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from tensorflow.python.keras.layers import Concatenate

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf', SGDClassifier(loss='hinge', penalty='l2', random_state=42, max_iter=50, tol=None)),
               ])

sgd.fit(X_train["extraction"], Y_train["meddra_term"])


y_pred = sgd.predict(X_test["extraction"])
print(classification_report(Y_test["meddra_term"], y_pred))
print('accuracy %s' % accuracy_score(y_pred, Y_test["meddra_term"]))

                                 precision    recall  f1-score   support

               MS-like syndrome       0.00      0.00      0.00         1
                  Schizophrenia       0.00      0.00      0.00         1
                abnormal dreams       0.00      0.00      0.00         0
                           ache       0.00      0.00      0.00         2
     achilles tendon discomfort       0.00      0.00      0.00         1
                aching in knees       0.00      0.00      0.00         0
                      addiction       0.00      0.00      0.00         2
            aggressive behavior       0.00      0.00      0.00         1
              allergic reaction       0.50      1.00      0.67         3
                        anxiety       1.00      1.00      1.00         1
                appetite absent       0.00      0.00      0.00         0
                  appetite lost       0.00      0.00      0.00         0
            appetite suppressed       0.00      0.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
# Rocchio
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', NearestCentroid()),
                     ])
text_clf.fit(X_train["extraction"], Y_train["meddra_term"])
predicted = text_clf.predict(X_test["extraction"])
print(metrics.classification_report(Y_test["meddra_term"], predicted))
print('accuracy %s' % accuracy_score(y_pred, Y_test["meddra_term"]))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.nearest_centroid module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


                                  precision    recall  f1-score   support

                MS-like syndrome       0.00      0.00      0.00         1
                   Schizophrenia       0.00      0.00      0.00         1
                            ache       0.00      0.00      0.00         2
      achilles tendon discomfort       0.00      0.00      0.00         1
                 aching in knees       0.00      0.00      0.00         0
                       addiction       0.00      0.00      0.00         2
             aggressive behavior       0.00      0.00      0.00         1
               allergic reaction       1.00      0.33      0.50         3
allergic reaction to antibiotics       0.00      0.00      0.00         0
                         anxiety       1.00      1.00      1.00         1
                 appetite absent       0.00      0.00      0.00         0
                   appetite lost       0.00      0.00      0.00         0
             appetite suppressed     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# Bagging
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', BaggingClassifier(KNeighborsClassifier())),
                     ])
text_clf.fit(X_train["extraction"], Y_train["meddra_term"])
predicted = text_clf.predict(X_test["extraction"])
print(metrics.classification_report(Y_test["meddra_term"], predicted))
print('accuracy %s' % accuracy_score(y_pred, Y_test["meddra_term"]))

                                  precision    recall  f1-score   support

                MS-like syndrome       0.00      0.00      0.00         1
                   Schizophrenia       0.00      0.00      0.00         1
                 abnormal dreams       0.00      0.00      0.00         0
                            ache       0.00      0.00      0.00         2
      achilles tendon discomfort       0.00      0.00      0.00         1
                       addiction       0.00      0.00      0.00         2
             aggressive behavior       0.00      0.00      0.00         1
               allergic reaction       0.50      1.00      0.67         3
                         anxiety       0.00      0.00      0.00         1
                   appetite lost       0.00      0.00      0.00         0
             appetite suppressed       0.00      0.00      0.00         3
      attention deficit disorder       0.00      0.00      0.00         1
              attention impaired     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
Y_train['meddra_term_category'] = labelencoder.fit_transform(Y_train['meddra_term'])
Y_test['meddra_term_category'] = labelencoder.fit_transform(Y_test['meddra_term'])

In [0]:
# from tensorflow.python.keras.layers import Dense
# from tensorflow.python.keras import Sequential
# from tensorflow.python.keras.layers import Embedding
# from tensorflow.python.keras.layers import Dropout
# from tensorflow.python.keras.layers import Activation
# from tensorflow.python.keras.layers import Convolution1D

In [0]:
# model = Sequential()
# model.add(Embedding(10000, 128))
# model.add(Convolution1D(nb_filter=250,
#                         filter_length=5,
#                         border_mode='valid',
#                         activation='relu',
#                         subsample_length=1))
# model.add(Dense(hidden_dims)) 
# model.add(Dropout(0.2)) 
# model.add(Activation('relu'))
# model.add(Dense(nb_classes))
# model.add(Activation('sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.fit(X_train["extraction"], Y_train["meddra_term"], batch_size=batch_size, nb_epoch=1,
#           validation_data=(X_test["extraction"], Y_test["meddra_term"]))

# preds = model.predict_classes(X_test["extraction"], verbose=0)

In [0]:
# print(metrics.classification_report(Y_test["meddra_term"], predicted))